oding: utf-8

SPDX-FileCopyrightText: 2021 Alfred-Wegener-Institut, Helmholtz-Zentrum für <br>
Polar- und Meeresforschung (AWI)<br>
SPDX-License-Identifier: MIT


Growth Model Atlantic Cod.<br>
Compute weight-at-age of Atlantic cod using multidimensional ocean <br>
temperature data in the Celtic Sea (47° - 52°N, -12°W - -1°W).<br>
Requirements:<br>
-------------<br>
    <br>
    numpy<br>
    pandas<br>
    xarray<br>
Usage:<br>
------<br>
    Execute on shell or interpreter:<br>
#        e.g.: [/usr/foo/pt_profiles] bar% python3 growth_model_transient.py<br>
               python growth_model_transient.py<br>
        <br>
Authors:<br>
------  <br>
mbutzin 2015 (first version) - 2018 (cleaned code)<br>
nsokolov 2018 - 2022<br>
arohner 2021 - 2022<br>


In [16]:
import os
import sys

import numpy as np
import pandas as pd
import xarray as xr

import growth_model_functions_constants
from growth_model_functions_constants import equation2, equation3
from supporting_functions import save_netcdf
import user_setup

Define geographic boundaries (latitudes, longitudes)<br>
Default coordinates: Celtic Sea coordinates

In [17]:
lat_coords = user_setup.lat_coords
lon_coords = user_setup.lon_coords

Define depth_levels for your growth model output files

In [18]:
depth_levels = user_setup.depth_levels 

Specify directory

In [19]:
output_dir= user_setup.output_directory 
print('My output directory: ', output_dir)

My output directory:  /Users/benjamin/growth_model_output/


In [20]:
folder_name = user_setup.experiment_name 
experiment_name = user_setup.region
file_extension = user_setup.file_extension

Define initial time step (month = 1)

In [21]:
dt = 1

Load constant parameters for the growth equation <br>
(values are from Eq.2 (Butzin and Poertner, 2016))

In [22]:
C_AVG = growth_model_functions_constants.C_AVG

decode_times=True or False, concat_dim=True or False

In [23]:
my_files = user_setup.my_files

Get temperature files from input directory<br>
You need to give: input directory, file_extension, decode_times=True or False, concat_dim=True or False

In [24]:
input_files = xr.open_mfdataset(my_files, decode_times=False)

Create new_time for the dataset to change it from 360_d to standard calendar<br>
Check your netcdf files which calender they have

In [25]:
new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')

You need new times in you dataset<br>
Only if you have other than standard calendar

In [26]:
input_files = input_files.assign_coords({'time': new_time})

Define latitudes for the output netcdf file

In [27]:
lat = user_setup.lat
lon = user_setup.lon

Define depths coordinates for the output netcdf file<br>
The depths range should represented the observed <br>
depth range of cod distribution

In [28]:
depths = user_setup.depths

Number of years in one life cycle of an individual cod species  

In [29]:
generation = user_setup.generation
# Should be one year less than starting year in the dataset
initial_year = user_setup.initial_year
# Number of years in the input temperature dataset
years = user_setup.years
# Define dimensionality of coords
N_depths = len(depths)
N_lat = len(lat)
N_lon = len(lon)

Start calculations

In [30]:
for each_year in years:
    # Selecting the inital year (birth of cod)         
    initial_year = initial_year + 1
    print('INITIAL YEAR: ', initial_year)
    
    # Identifiyng the last year (death of cod)
    last_year = initial_year + generation         
    print('LAST YEAR: ', last_year)
    
    # Beginning of life cycle
    age = 0                                       
    
    weight = np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

    for every_year in range(initial_year, last_year):
        if last_year > years[-1]: print('Done')   ;break                                       
        else:
            print(every_year)
            my_temp = input_files.thetao.sel(time=str(every_year), 
            depth_coord = depth_levels, latitude = lat_coords, 
            longitude = lon_coords)            
            print('WORK ON YEAR: ', every_year)

            # Partly vectorize 4D temperature fields 
            # to accelerate the computations
            temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             
               
            # Set NaN values                         
          
            temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan     

            # Initialize necessary variable fields
            a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
            b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
            if every_year == initial_year: mm0 = 2           
            else: mm0 = 0
            for mon in np.arange(0,12):
              for dd in np.arange(0,30):
                for ilev in np.arange(0, N_depths):  
                  a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
                  b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
                  growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :]** b[ilev, :] - C_AVG )  
                  # for i in growth_rates[ilev]:
                  #   if i < 0:
                  #     print(mon,dd,ilev,i)
                  
                  
                  growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0,0, growth_rates[ilev, :])
                  weight[ilev, :] = weight[ilev, :] * (1. + dt * growth_rates[ilev, :])
                                                                
            new_year = int(every_year) + 1
            
            # Reshape data to original shape
            a_3d = a.reshape(N_depths, N_lat, N_lon)
            b_3d = b.reshape(N_depths, N_lat, N_lon)
            growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)
            
            # 3D field with asymptotic weight
            weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
            
            # Calculate maximum asymptotic weight at a given location 
            # ("W*" in Butzin and Pörtner (2016)) 
            weight_max = np.nanmax(weight_3d, axis = 0)
            
           
            
            # Create output directories if they do not exist
            
            # OUT = output_dir + folder_name + '/init_' + str(initial_year) +   \
            #       '_transient/'+experiment_name 
            
            # output_a_3d = OUT + '/a_3d/'
            
            # output_b_3d = OUT + '/b_3d/'
            
            # output_growth_rates_3d = OUT + '/growth_rates/'
            
            # output_weight_3d = OUT + '/weight_3d/'
            
            # output_weight_max = OUT + '/weight_max/'
            
            # for F in [output_a_3d, output_b_3d, output_growth_rates_3d, 
            #           output_weight_3d, output_weight_max]:
                
            #     if not os.path.exists(F): os.makedirs(F)
                
            # print(OUT)   

            #             # Save netcdf
            # save_netcdf(a_3d, 'a_3d', output_a_3d, new_year, age)
                                    
            # save_netcdf(b_3d, 'b_3d', output_b_3d, new_year, age)
                                 
            
            # save_netcdf(growth_rates_3d, 'growth_rates_3d', 
            #             output_growth_rates_3d, new_year, age)
                                                           
            
            # save_netcdf(weight_3d, 'weight_3d', output_weight_3d, new_year, 
            #             age)
                                           
            
            # # Save weight_max           
            # save_netcdf(weight_max, 'weight_max', output_weight_max, new_year, 
            #             age, dimension_labels = ('latitude', 'longitude'))

INITIAL YEAR:  2000
LAST YEAR:  2002
2000
WORK ON YEAR:  2000


/var/folders/j1/k2clnjz530q64v44gw4srhpw0000gn/T/ipykernel_93527/1697752640.py:64: RuntimeWarning: All-NaN slice encountered
  weight_max = np.nanmax(weight_3d, axis = 0)


2001
WORK ON YEAR:  2001
INITIAL YEAR:  2001
LAST YEAR:  2003
2001
WORK ON YEAR:  2001
2002
WORK ON YEAR:  2002
INITIAL YEAR:  2002
LAST YEAR:  2004
2002
WORK ON YEAR:  2002
2003
WORK ON YEAR:  2003
INITIAL YEAR:  2003
LAST YEAR:  2005
Done
INITIAL YEAR:  2004
LAST YEAR:  2006
Done
